In [21]:
import os
import pickle

with open("processed_stories.pkl", "rb") as processed_pickle_file:
    master_list = pickle.load(processed_pickle_file)

In [22]:

# 2.1 clean story and summaries, convert to lowercase, remove punctuation, remove digits
import string

for item in master_list:
    item["story"] = item["story"].lower().replace('(cnn)', "").translate(str.maketrans('', '', string.punctuation)).translate(str.maketrans('', '', string.digits))
    for highlight_index, _ in enumerate(item["highlights"]):
        item["highlights"][highlight_index] = item["highlights"][highlight_index].lower().translate(str.maketrans('', '', string.punctuation)).translate(str.maketrans('', '', string.digits))

In [38]:
def get_sentences_from_story(story: str):
    return filter(lambda s: s.strip() != "" , story.split("\n"))

In [35]:
import itertools
from rouge import Rouge 

rouge = Rouge(["rouge-1"])

def calculate_top_rouge_score_for_story_summary_pair(story: str, summary: list, index: int):
    story_sentances = get_sentences_from_story(story)
    pairings = itertools.product(story_sentances, summary)
    sentance_scores = {}
    for pairing in pairings:
        scores = rouge.get_scores(pairing[0], pairing[1])
        sentance_scores[pairing[0]] = scores[0]["rouge-1"]["f"]
    
    sorted_sentance_scores = sorted(sentance_scores.items(), key=lambda x: x[1], reverse=True)
    list_index = min(len(sorted_sentance_scores), 5)
    if list_index == 0:
        return [{"": 0}]
    return list(sorted_sentance_scores[:list_index])

for index, item in enumerate(master_list):
    print(index)
    item["top_sentance_summaries"] = calculate_top_rouge_score_for_story_summary_pair(item["story"], item["highlights"], index)

5
89246
89247
89248
89249
89250
89251
89252
89253
89254
89255
89256
89257
89258
89259
89260
89261
89262
89263
89264
89265
89266
89267
89268
89269
89270
89271
89272
89273
89274
89275
89276
89277
89278
89279
89280
89281
89282
89283
89284
89285
89286
89287
89288
89289
89290
89291
89292
89293
89294
89295
89296
89297
89298
89299
89300
89301
89302
89303
89304
89305
89306
89307
89308
89309
89310
89311
89312
89313
89314
89315
89316
89317
89318
89319
89320
89321
89322
89323
89324
89325
89326
89327
89328
89329
89330
89331
89332
89333
89334
89335
89336
89337
89338
89339
89340
89341
89342
89343
89344
89345
89346
89347
89348
89349
89350
89351
89352
89353
89354
89355
89356
89357
89358
89359
89360
89361
89362
89363
89364
89365
89366
89367
89368
89369
89370
89371
89372
89373
89374
89375
89376
89377
89378
89379
89380
89381
89382
89383
89384
89385
89386
89387
89388
89389
89390
89391
89392
89393
89394
89395
89396
89397
89398
89399
89400
89401
89402
89403
89404
89405
89406
89407
89408
89409
89410
89411
89

In [51]:
# 2.3 create pandas dataframe
# https://www.geeksforgeeks.org/create-a-pandas-dataframe-from-list-of-dicts/
# For each story, for each sentence in a story
# Check if the sentence exists in the summary list extracted in the previous step for the corresponding story.
# If the sentence exists, store it in a list and provide the sentence a label of 1. Otherwise, the sentence gets a label of 0, but still gets stored in a list.
# Create a pandas dataframe with the lists created in the above step.
# Serialize the dataframe in .pkl format for later use.

import pandas as pd

intermediate_sentances = {"story_id": [], "sentence_id": [], "sentence": [], "candidate": []}
for index, item in enumerate(master_list):
    print(index)
    if not isinstance(item["top_sentance_summaries"][0], dict):
        summary_candidates = list([pair[0] for pair in item["top_sentance_summaries"]])
    else:
        summary_candidates = {}
    story_sentences = get_sentences_from_story(item["story"])
    for sentance_id, sentence in enumerate(story_sentences):
        intermediate_sentances["story_id"].append(item["story_id"])
        intermediate_sentances["sentence_id"].append(sentance_id)
        intermediate_sentances["sentence"].append(sentence)
        intermediate_sentances["candidate"].append(int(sentence in summary_candidates))




5
89246
89247
89248
89249
89250
89251
89252
89253
89254
89255
89256
89257
89258
89259
89260
89261
89262
89263
89264
89265
89266
89267
89268
89269
89270
89271
89272
89273
89274
89275
89276
89277
89278
89279
89280
89281
89282
89283
89284
89285
89286
89287
89288
89289
89290
89291
89292
89293
89294
89295
89296
89297
89298
89299
89300
89301
89302
89303
89304
89305
89306
89307
89308
89309
89310
89311
89312
89313
89314
89315
89316
89317
89318
89319
89320
89321
89322
89323
89324
89325
89326
89327
89328
89329
89330
89331
89332
89333
89334
89335
89336
89337
89338
89339
89340
89341
89342
89343
89344
89345
89346
89347
89348
89349
89350
89351
89352
89353
89354
89355
89356
89357
89358
89359
89360
89361
89362
89363
89364
89365
89366
89367
89368
89369
89370
89371
89372
89373
89374
89375
89376
89377
89378
89379
89380
89381
89382
89383
89384
89385
89386
89387
89388
89389
89390
89391
89392
89393
89394
89395
89396
89397
89398
89399
89400
89401
89402
89403
89404
89405
89406
89407
89408
89409
89410
89411
89

In [44]:
sentence_frame = pd.DataFrame(intermediate_sentances)

In [50]:
sentence_frame.to_pickle("sentence_frame.pkl")

with open("intermediate_master_list.pkl", "wb") as master_list_file:
    pickle.dump(master_list, master_list_file)

with open("intermediate_sentances.pkl", "wb") as intermediate_sentances_file:
    pickle.dump(intermediate_sentances, intermediate_sentances_file)